#Классификация отзывов на положительные и отрицательные
Ноутбук для соревнования по ИАД 

Итоговый скор в соревновании > 0.99


In [0]:
import numpy as np
import tensorflow as tf

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string     
import re


from IPython.display import Image
from IPython.core.display import HTML 

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

In [0]:
from keras.layers import Dense, Input, GRU, Embedding, Dropout, Bidirectional, LSTM
from keras.models import Model, Sequential
from keras.optimizers import Adam
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, LearningRateScheduler
from tqdm import tqdm_notebook

Using TensorFlow backend.


 Загрузка данных. Был использован kaggle api, поэтому в директорию нужно положить json файл

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
! chmod 600 /root/.kaggle/kaggle.json
import kaggle

In [0]:
! kaggle competitions download -c iad-deep-learning-sentiment

  0% 0.00/8.37M [00:00<?, ?B/s]
100% 8.37M/8.37M [00:00<00:00, 77.4MB/s]
 99% 589M/595M [00:11<00:00, 163MB/s]
100% 595M/595M [00:11<00:00, 56.1MB/s]
  0% 0.00/2.08M [00:00<?, ?B/s]
100% 2.08M/2.08M [00:00<00:00, 141MB/s]
 72% 49.0M/68.0M [00:02<00:01, 18.3MB/s]
100% 68.0M/68.0M [00:02<00:00, 24.4MB/s]


In [0]:
! unzip "x_train.txt.zip"
! unzip "x_test.txt.zip"
! unzip "y_train.csv.zip"
! unzip "random_prediction.csv.zip"


Archive:  x_train.txt.zip
  inflating: x_train.txt             
Archive:  x_test.txt.zip
  inflating: x_test.txt              
Archive:  y_train.csv.zip
  inflating: y_train.csv             
Archive:  random_prediction.csv.zip
  inflating: random_prediction.csv   


In [0]:
# функция для создания датасета
def make_data(text):


  dataset = {'review':[]}
  with open(text, 'r') as f:
    for review in f.readlines():
      dataset['review'].append(review)
  data = pd.DataFrame.from_dict(dataset)
  return data

In [0]:
data_train = make_data('x_train.txt')
data_test = make_data('x_test.txt')

In [0]:
y = pd.read_csv('y_train.csv')

# Подготовка и очистка текста
Tекст очищен от пунктуации и приведен к нижнему регистру. Функция работает достаточно долго, поэтому к данным можно как применить функцию, так и загрузить уже подготовленные данные с дропбокс с помощью wget



In [0]:
def clean_and_prepare(x_train):


  def punc_and_register(x):


    punctuation = list(string.punctuation)
    return ''.join([a.lower() if a not in punctuation + ['\n'] else ' ' for a in x])

  x_train['review'].apply(punc_and_register)
  return x_train

In [0]:
#x_train = clean_and_prepare(data_train)
#x_test = clean_and_prepare(data_test)

In [0]:
#x_train.to_csv('x_train_clean.csv')
#x_test.to_csv('x_test_clean.csv')

In [0]:
! wget https://www.dropbox.com/s/8clqqvond56esz0/x_train_clean.csv?dl=0 -O "x_train_clean.csv"
! wget https://www.dropbox.com/s/0pxxvva12zjl7g8/x_test_clean.csv?dl=0 -O "x_test_clean.csv"

--2019-12-23 09:03:54--  https://www.dropbox.com/s/8clqqvond56esz0/x_train_clean.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.82.1, 2620:100:6032:1::a27d:5201
Connecting to www.dropbox.com (www.dropbox.com)|162.125.82.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/8clqqvond56esz0/x_train_clean.csv [following]
--2019-12-23 09:03:54--  https://www.dropbox.com/s/raw/8clqqvond56esz0/x_train_clean.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc0e175e88bb304a7c42e78366e3.dl.dropboxusercontent.com/cd/0/inline/AuwnzXSj3N3tCsd4IyvnyNv-otXKqwF1YcB62zEcGr8BOZGuQBKp6Q_8FaSkUTnlwENM6c2msBPfCKCjqcAsOxRTJM__FIYQB4Gj3KxYaG_UAjZOmEKeESOkMQn5ZiaKmiE/file# [following]
--2019-12-23 09:03:55--  https://uc0e175e88bb304a7c42e78366e3.dl.dropboxusercontent.com/cd/0/inline/AuwnzXSj3N3tCsd4IyvnyNv-otXKqwF1YcB62zEcGr8BOZGuQBKp6Q_8FaSkUTnlwENM6c2msBPfCKCjqcAsOxRTJM_

In [0]:
x_train = pd.read_csv('x_train_clean.csv')
x_test = pd.read_csv('x_test_clean.csv')

In [0]:
x_train.head()

,Unnamed: 0,review
0,0,stuning even for the non gamer this sound tra...
1,1,the best soundtrack ever to anything i m rea...
2,2,amazing this soundtrack is my favorite music...
3,3,excellent soundtrack i truly like this soundt...
4,4,remember pull your jaw off the floor after he...


In [0]:
x_train = x_train['review']
x_test = x_test['review']

Загрузкa Glove эмбедингов. Были использованы и другие, но качество было хуже, поэтому оставляем именно их.

In [0]:
!wget http://nlp.stanford.edu/data/wordvecs/glove.42B.300d.zip

--2019-12-14 15:21:59--  http://nlp.stanford.edu/data/wordvecs/glove.42B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/wordvecs/glove.42B.300d.zip [following]
--2019-12-14 15:21:59--  https://nlp.stanford.edu/data/wordvecs/glove.42B.300d.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/wordvecs/glove.42B.300d.zip [following]
--2019-12-14 15:22:00--  http://downloads.cs.stanford.edu/nlp/data/wordvecs/glove.42B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Le

In [0]:
! unzip "glove.42B.300d.zip"

Archive:  glove.42B.300d.zip
  inflating: glove.42B.300d.txt      


In [0]:
f = open('./glove.42B.300d.txt')

embeddings_index = dict() 
for line in tqdm_notebook(f):
    values = line.split(' ')
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

 #  Обрабатываем датасет для подачи в нейронку

 Добавляем padding и создаем матрицу эмбеддингов, если для слова нет вектора  - добавляем его во множество неизвестных слов

In [0]:

embed_size = 300 # how big is each word vector
max_features = 100000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 150 # max number of words in a comment to use

list_sentences_train = x_train.values
y = y.Probability.values
list_sentences_test = x_test.values

tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
x_train_pad = pad_sequences(list_tokenized_train, maxlen=maxlen)
x_test_pad = pad_sequences(list_tokenized_test, maxlen=maxlen)

In [0]:
all_embs = np.stack(embeddings_index.values())
emb_mean, emb_std = all_embs.mean(), all_embs.std()
emb_mean, emb_std

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  if self.run_code(code, result):


(0.005720101, 0.2951066)

In [0]:
word_index = tokenizer.word_index
unknown_words = set()
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector
    else: unknown_words.add(word)

# Создаем модель

Собственно сама архитектура сети: очень похожа на ту, которая использовалась на семинаре
* Слой эмеддингов
* 2 слоя bidirectional LSTM - так как в целом LSTM считается мощнее, чем GRU, хотя и обучается больше (размер вектора памяти h - 264)
* После каждого слоя применен Dropout (0.4 и 0.3 соответственно)
* Полносвязный слой с 128 нейронами и сигмоидой
* Выход классификатора с одним нейроном, показывающий вероятность принадлежности объекта к положительному классу 

Остальные параметры обучения: батч побольше 1024, метод оптимизации Адам с  lr scheduler.

Модель обучалась 6 эпох.

In [0]:
input_layer = Input((maxlen,), name = 'reviews')
embedding_layer = Embedding(max_features, embed_size, input_length=maxlen, 
                            weights=[embedding_matrix], 
                            trainable = False)(input_layer)
x = Bidirectional(LSTM(264, return_sequences=True))(embedding_layer)
x = Dropout(0.4)(x)
x = Bidirectional(LSTM(264, return_sequences=False))(x)
x = Dropout(0.3)(x)
x = Dense(128, activation="relu")(x)
output_layer = Dense(1, activation="sigmoid")(x)

model = Model(inputs=input_layer, outputs=output_layer)
model.compile(loss='binary_crossentropy',
                  optimizer=Adam(clipvalue=1, clipnorm=1),
                  metrics=['accuracy'])                                                                               
print(model.summary())

def schedule(ind):
    a = [0.001, 0.001, 0.0001, 0.0001, 0.00001, 0.00001, 0.000001]
    return a[ind]

lr = LearningRateScheduler(schedule)
    
early_stop = EarlyStopping(monitor='val_loss',
                           patience=4,
                           verbose=1,
                           min_delta=1e-4)


history = model.fit(x_train_pad, y, batch_size=1024, epochs = 3, 
                    validation_split = 0.1, verbose = 1, callbacks = [lr]) #decay rate?????











Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reviews (InputLayer)         (None, 150)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 150, 300)          30000000  
_________________________________________________________________
bidirectional_1 (Bidirection (None, 150, 528)          1193280   
_________________________________________________________________
dropout_1 (Dropout)          (None, 150, 528)          0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 528)               1674816   
____________________________________

In [0]:
history = model.fit(x_train_pad, y, batch_size=1024, epochs = 3, 
                    validation_split = 0.1, verbose = 1, callbacks = [lr])

Train on 3240000 samples, validate on 360000 samples
Epoch 1/3
3240000/3240000 [==============================] - 3521s 1ms/step - loss: 0.1059 - acc: 0.9620 - val_loss: 0.1109 - val_acc: 0.9600
Epoch 2/3
3240000/3240000 [==============================] - 3518s 1ms/step - loss: 0.0964 - acc: 0.9659 - val_loss: 0.1105 - val_acc: 0.9604
Epoch 3/3
3240000/3240000 [==============================] - 3517s 1ms/step - loss: 0.0768 - acc: 0.9740 - val_loss: 0.1131 - val_acc: 0.9615


Результат эксперимента с этой моделью оказался очень неплохим. Видно, что она переобучается, но качество более-менее. Амбицией было разобраться с BERT для улучшения качества, но как-то не вышло:( поэтому оставляю так

# Создаем файл с предсказанием

In [0]:
sample = pd.read_csv("random_prediction.csv")

In [0]:
sample.head()

,Id,Probability
0,1,0.24916
1,2,0.47618
2,3,0.26317
3,4,0.29403
4,5,0.49895


In [0]:
#работает доолго, но попробовать обернуть данные в тф.датасет, к сожалению, не успела(())
prediction = model.predict(x_test_pad)

In [0]:
sample['Probability'] = prediction
sample.to_csv('Sub2', index=False)

In [0]:
sample.head()

,Id,Probability
0,1,0.996444
1,2,0.997371
2,3,0.000508
3,4,0.438972
4,5,0.997879
